In [ ]:
from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM, Bidirectional, Embedding, Dropout
from keras.utils.vis_utils import plot_model
from keras.utils.np_utils import to_categorical
from keras.utils import np_utils
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
from sklearn import metrics
from math import sqrt


In [ ]:
import pandas as pd
#max data_points 225745
data_points = 10000

pve = pd.read_csv('dataset_attack.csv', nrows=data_points).drop(['frame.encap_type', 'ip.src', 'ip.dst','frame.protocols'],axis=1)
nve = pd.read_csv('dataset_normal.csv', nrows=data_points).drop(['frame.encap_type','ip.src', 'ip.dst','frame.protocols'],axis=1)
print ("attack set", pve.head)
print ("normal set", nve.head)

In [ ]:
import numpy as ny

X_P= pve.drop(['class'], axis = 1).values
X_N= nve.drop(['class'], axis = 1).values
y_p= pve['class'].replace(to_replace="attack",value="1")
y_n= nve['class'].replace(to_replace="normal",value="0")
X=ny.concatenate((X_P,X_N))
y=ny.concatenate((y_p,y_n))
print(X)
print(y)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X,y)
clf = RandomForestClassifier(n_estimators=100, max_depth=20)
clf.fit(x_train,y_train)
y_pred = clf.predict(x_test)
cm = confusion_matrix(y_test, y_pred)
acc = clf.score(x_test, y_test)
rmse = sqrt(mean_squared_error(y_test, y_pred))
print("Random Forest")
print ("Accuracy : ", acc)
print("Confusion Matrix")
print (cm)
summary = classification_report(y_test, y_pred)
plot_confusion_matrix(clf, x_test, y_test)
print(summary)
cross_val_scr = cross_val_score(clf, X, y, cv=10,scoring='accuracy')
print("CV Score")
print(cross_val_scr)
print(" Mean: {}".format(ny.mean(cross_val_scr)))
plt.show()


In [ ]:
col_cnt = X.shape[1]
row_cnt = X.shape[0]
matrix = ny.zeros((row_cnt-10, 10, 25))
for i in range(row_cnt-10):
    temp = ny.zeros((10, 25))
    for j in range(i, i + 10 - 1):
        temp[j-i] = X[j]
    matrix[i] = temp
#print(I)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(matrix, y[10:row_cnt], test_size = 0.2)
y_train=ny.asarray(y_train).astype(ny.int32)
print(X_train)
print(y_train)

In [ ]:
def create_model():
    model = Sequential()
    model.add(Bidirectional(LSTM(64, activation='tanh', kernel_regularizer='l2')))
    model.add(Dense(128, activation = 'relu', kernel_regularizer='l2'))
    model.add(Dense(1, activation = 'sigmoid', kernel_regularizer='l2'))
    return model

In [ ]:
model = create_model()
model.compile(loss='mse', optimizer="rmsprop", metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs = 10,validation_split=0.2, verbose = 1)